In [49]:
%reload_ext autoreload
%autoreload 2
from openfl.interface.interactive_api.federation import Federation
from openfl.interface.interactive_api.experiment import ModelInterface, FLExperiment
from openfl.utilities.optimizers.torch import FedProxOptimizer, FedProxAdam
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
import numpy as np
from gear_shard_dataset import GearSD
from kvasir_shard_dataset import KvasirSD

# Vs Code problem not reloading correctly python module
import importlib
import loss
import models
import tasks
importlib.reload(loss)
importlib.reload(models)
importlib.reload(tasks)

from loss import *
from models import *
from tasks import Task

import matplotlib.pyplot as plt
import time
import os
import copy


NUM_CLASSES=4
ROUND_TO_TRAIN=10

client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port = 50055
experiment_name = 'gear_test_experiment'
LEARNING_RATE=1e-3
TRAIN_BS=16
VALID_BS=8

# back ground - conveyor - good - notgood
weights = [0.5, 0.5, 1, 1]
class_weights = torch.FloatTensor(weights)
CRITERION=nn.CrossEntropyLoss(weight=class_weights)
CRITERION_VAL=mIOU

# Once your model is trained, the predict function will outputs a (400,400,num_classes) 
# mask with probability inside it.


## don't forget to launch envoy service
### bash start_envoy.sh env_on localhost

In [50]:
# please use the same identificator that was used in signed certificate
federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port,
    tls=False
)
shard_registry = federation.get_shard_registry()
shard_registry
federation.target_shape
dummy_shard_desc = federation.get_dummy_shard_descriptor(size=10)
dummy_shard_dataset = dummy_shard_desc.get_dataset('train')
sample, target = dummy_shard_dataset[0]
f"Sample shape: {sample.shape}, target shape: {target.shape}"

#fed_dataset = KvasirSD(train_bs=TRAIN_BS, valid_bs=VALID_BS)
fed_dataset = GearSD(num_classes=4, train_bs=TRAIN_BS, valid_bs=VALID_BS)

fed_dataset.shard_descriptor = dummy_shard_desc
for i, (sample, target) in enumerate(fed_dataset.get_train_loader()):
    print("Sample shape : "+str(sample.shape))
    print("Target shape : "+str(target.shape))

[*] Dataset len 10
Sample shape : torch.Size([9, 3, 400, 400])
Target shape : torch.Size([9, 400, 400])


In [51]:
## Load previous model or build an initial one
d = DeepLabv3()
model, params_to_update= d.build_deeplab(NUM_CLASSES, alpha=0, keep_features=True)
#model = d.build_deeplab(load_from_pkl=True)

# take low learning rate for Tversky loss and to not change so much the current trained weights
optimizer_adam = optim.Adam(params_to_update, lr=LEARNING_RATE)
# we can try SGD
optimizer = optim.SGD(params_to_update, lr=LEARNING_RATE, momentum=0.9)


[*] Building model ..
[*] Keeping features of deeplabv3, freezing all the model except the head
198 layers in this model, freezing 198 layer

[*] Changing head for 4 classes and removing aux classifier
[*] Done
Params to learn:
	 classifier.0.convs.0.0.weight
	 classifier.0.convs.0.1.weight
	 classifier.0.convs.0.1.bias
	 classifier.0.convs.1.0.weight
	 classifier.0.convs.1.1.weight
	 classifier.0.convs.1.1.bias
	 classifier.0.convs.2.0.weight
	 classifier.0.convs.2.1.weight
	 classifier.0.convs.2.1.bias
	 classifier.0.convs.3.0.weight
	 classifier.0.convs.3.1.weight
	 classifier.0.convs.3.1.bias
	 classifier.0.convs.4.1.weight
	 classifier.0.convs.4.2.weight
	 classifier.0.convs.4.2.bias
	 classifier.0.project.0.weight
	 classifier.0.project.1.weight
	 classifier.0.project.1.bias
	 classifier.1.weight
	 classifier.2.weight
	 classifier.2.bias
	 classifier.4.weight
	 classifier.4.bias


In [52]:

framework_adapter = 'openfl.plugins.frameworks_adapters.pytorch_adapter.FrameworkAdapterPlugin'
MI = ModelInterface(model=model, optimizer=optimizer, framework_plugin=framework_adapter)
# Save the initial model state
initial_model = deepcopy(model)

TI, validate = Task.createTask(CRITERION, CRITERION_VAL, NUM_CLASSES)

# create an experimnet in federation
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=fed_dataset,
                    rounds_to_train=ROUND_TO_TRAIN,
                    opt_treatment='CONTINUE_GLOBAL',
                    device_assignment_policy='CUDA_PREFERRED')

### You may use the same federation object to report another experiment or even schedule several experiments that will be executed in series.

# Stream from tensorboard


In [ ]:
# we can stream metrics 
fl_experiment.stream_metrics()
!tensorboard --logdir=logs

## Save your model and validate

In [43]:
best_model = fl_experiment.get_best_model()
torch.save(best_model, "save/best_model.pkl")


/home/cloud/anaconda3/lib/python3.8/site-packages/openfl/plugins/frameworks_adapters/pytorch_adapter.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


In [ ]:

best_model = fl_experiment.get_best_model()
# we can also retreive last model with get_last_model()

# We remove exremove_experiment_datamove_experiment_datamove_experiment_datariment data from director
fl_experiment.remove_experiment_data()

# Compare initial model 
validate(soft_dice_loss, soft_dice_coef, initial_model, fed_dataset.get_valid_loader(), 'cpu')

# With the best model 
validate(soft_dice_loss, soft_dice_coef, best_model, fed_dataset.get_valid_loader(), 'cpu')

# We can save the best and use it on runtime 
# TODO save ..

# We can also improve the model 

In [17]:
MI = ModelInterface(model=best_model, optimizer=optimizer_adam, framework_plugin=framework_adapter)
fl_experiment.start(model_provider=MI, task_keeper=TI, data_loader=fed_dataset, rounds_to_train=8, \
                              opt_treatment='CONTINUE_GLOBAL')

# optimizer treatment : RESET: the optimizer state is initialized each round from noise
#CONTINUE_LOCAL: the optimizer state will be reused locally by every collaborator
#CONTINUE_GLOBAL: the optimizer’s state will be aggregated 

## We can create specific task 

In [ ]:
tasks = task_keeper.get_registered_tasks()

In [14]:
def filter_assigner(collaborators, round_number, **kwargs):
    collaborator_task_map = {}
    exclude_collaborators = ['env_two', 'env_three']
    for collaborator_name in collaborators:
        if collaborator_name in exclude_collaborators:
            continue
        collaborator_task_map[collaborator_name] = [
            tasks['train'],
            tasks['locally_tuned_model_validate'],
            tasks['aggregated_model_validate']
        ]
    return collaborator_task_map

In [ ]:
def random_assigner(collaborators, round_number, **kwargs):
    """Assigning task groups randomly while ensuring target distribution"""
    import random
    random.shuffle(collaborators)
    collaborator_task_map = {}
    for idx, col in enumerate(collaborators):
        # select only 70% collaborators for training and validation, 30% for validation
        if (idx+1)/len(collaborators) <= 0.7:
            collaborator_task_map[col] = tasks.values()  # all three tasks
        else:
            collaborator_task_map[col] = [tasks['aggregated_model_validate']]
    return collaborator_task_map

# exclude collaborator

shard_registry = federation.get_shard_registry()
def filter_by_shard_registry_assigner(collaborators, round_number, **kwargs):
    collaborator_task_map = {}
    for collaborator in collaborators:
        col_status = shard_registry.get(collaborator)
        if not col_status or not col_status['is_online']:
            continue
        node_info = col_status['shard_info'].node_info
        # Assign train task if collaborator has GPU with total memory more that 8 GB
        if len(node_info.cuda_devices) > 0 and node_info.cuda_devices[0].memory_total > 8 * 1024**3:
            collaborator_task_map[collaborator] = [
                tasks['train'],
                tasks['locally_tuned_model_validate'],
                tasks['aggregated_model_validate'],
            ]
        else:
            collaborator_task_map[collaborator] = [
                tasks['aggregated_model_validate'],
            ]
    return collaborator_task_map

In [ ]:
## Additional vlaidation round

rounds_to_train = 3
total_rounds = rounds_to_train + 1 # use fl_experiment.start(..., rounds_to_train=total_rounds,...)

def assigner_with_last_round_validation(collaborators, round_number, **kwargs):
    collaborator_task_map = {}
    for collaborator in collaborators:
        if round_number == total_rounds - 1:
            collaborator_task_map[collaborator] = [
                tasks['aggregated_model_validate'],
            ]
        else:
            collaborator_task_map[collaborator] = [
                tasks['train'],
                tasks['locally_tuned_model_validate'],
                tasks['aggregated_model_validate']
            ]
    return collaborator_task_map

# Inference

In [ ]:
model = torch.load("save/best_model.pkl")

im = cv2.imread("2022-04-21__15_02_30.bmp")
im_s = cv2.imread("2022-04-21__15_02_30_label_ground-truth_semantic.png")


model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x) 